<a href="https://colab.research.google.com/github/renatachavezm22/Regresion-No-Lineal-y-Regresion-Logistica/blob/main/Actividad_7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
from datetime import date, time, datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
Gastos = pd.read_csv("Gastos (SN).csv", index_col='Unnamed: 0')

## Eliminar Outliers

En este punto lo que se hizo fue generar una función que nos permitiera eliminar los valores atípicos de las funciones por el método de los quartiles. Se utilizó este método debido a que es el más estricto y que elimina la mayor cantidad de valores a diferencia del que es por varianza. 

Posteriormente, se aplicó la función a todos los DataFrame que se habían limpiado de valores nulos en actividades anteriores. La razón por la cual se eliminaron los valores atípicos, es para poder evitar que al momento de generar toda la estadística descriptiva, no se llegue a ver demasiado sesgada por este tipo de datos. 

In [ ]:
def outliers(df):
  numericas = df._get_numeric_data()
  y=numericas

  percentile25=y.quantile(0.25) 
  percentile75=y.quantile(0.75) 
  iqr= percentile75 - percentile25

  Limite_Superior_iqr= percentile75 + 1.5*iqr
  Limite_Inferior_iqr= percentile25 - 1.5*iqr

  data4_iqr= numericas[(y<=Limite_Superior_iqr)&(y>=Limite_Inferior_iqr)]

  data5_iqr=data4_iqr.copy()
  data5_iqr=data5_iqr.fillna(round(data4_iqr.mean(),1))
  data5_iqr.isnull().sum()

  categoricas = df.drop(data5_iqr.columns, axis = 1)
  df1 = pd.concat([categoricas, data5_iqr], axis = 1)
  return df1

In [ ]:
Gastos = outliers(Gastos)

##Funciones

In [ ]:
Gastos1 = Gastos.copy()
Gastos['FECHA'] = pd.to_datetime(Gastos['FECHA'])
Gastos1['año'] = Gastos['FECHA'].dt.year

In [ ]:
Compras = Gastos1[Gastos1['TIPO_GASTO']=='COMPRAS']
Otros = Gastos1[Gastos1['TIPO_GASTO']!='COMPRAS']
Otros['TIPO_GASTO'] = 'OTROS GASTOS'
Gastos1 = pd.concat([Compras, Otros], axis = 0)

<ipython-input-53-83694b318125>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Otros['TIPO_GASTO'] = 'OTROS GASTOS'


In [ ]:
Gastos2020 = pd.DataFrame()
Gastos2021 = pd.DataFrame()
Gastos2022 = pd.DataFrame()
Gastos2023 = pd.DataFrame()
Gastos2020 = Gastos1[Gastos1['año']== 2020]
Gastos2021 = Gastos1[Gastos1['año']== 2021]
Gastos2022 = Gastos1[Gastos1['año']== 2022]
Gastos2023 = Gastos1[Gastos1['año']== 2023]


In [ ]:
def RegLogistica (df):
  Vars_Indep= df[['TOTAL']]
  Var_Dep= df['TIPO_GASTO']
  X= Vars_Indep
  y= Var_Dep
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
  escalar = StandardScaler()
  X_train = escalar.fit_transform(X_train)
  X_test = escalar.transform(X_test)  
  algoritmo = LogisticRegression()
  algoritmo.fit(X_train, y_train)
  y_pred = algoritmo.predict(X_test) 
  matriz = confusion_matrix(y_test, y_pred)
  print('Matriz de Confusión:')
  print(matriz)
  precision = precision_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
  print('Precisión del modelo:', precision)
  exactitud = accuracy_score(y_test, y_pred)
  print('Exactitud del modelo:', exactitud)
  sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
  print('Sensibilidad del modelo:', sensibilidad)
  puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
  print('Puntaje F1 del modelo:', puntajef1)
  return (precision, exactitud, sensibilidad)

In [ ]:
Lprecision = []
Lexactitud = []
Lsensibilidad = []
precision, exactitud, sensibilidad = RegLogistica(Gastos2020)
Lprecision.append(precision)
Lexactitud.append(exactitud)
Lsensibilidad.append(sensibilidad)
precision, exactitud, sensibilidad = RegLogistica(Gastos2021)
Lprecision.append(precision)
Lexactitud.append(exactitud)
Lsensibilidad.append(sensibilidad)
precision, exactitud, sensibilidad = RegLogistica(Gastos2022)
Lprecision.append(precision)
Lexactitud.append(exactitud)
Lsensibilidad.append(sensibilidad)
precision, exactitud, sensibilidad = RegLogistica(Gastos2023)
Lprecision.append(precision)
Lexactitud.append(exactitud)
Lsensibilidad.append(sensibilidad)

Matriz de Confusión:
[[ 61 389]
 [ 41 512]]
Precisión del modelo: 0.5980392156862745
Exactitud del modelo: 0.5712861415752741
Sensibilidad del modelo: 0.13555555555555557
Puntaje F1 del modelo: 0.22101449275362323
Matriz de Confusión:
[[154 316]
 [101 340]]
Precisión del modelo: 0.6039215686274509
Exactitud del modelo: 0.5422612513721186
Sensibilidad del modelo: 0.3276595744680851
Puntaje F1 del modelo: 0.42482758620689653
Matriz de Confusión:
[[ 49 327]
 [ 35 363]]
Precisión del modelo: 0.5833333333333334
Exactitud del modelo: 0.5322997416020672
Sensibilidad del modelo: 0.13031914893617022
Puntaje F1 del modelo: 0.21304347826086958
Matriz de Confusión:
[[10 47]
 [ 5 58]]
Precisión del modelo: 0.6666666666666666
Exactitud del modelo: 0.5666666666666667
Sensibilidad del modelo: 0.17543859649122806
Puntaje F1 del modelo: 0.2777777777777778


In [ ]:
tabla = pd.DataFrame()
anios = ['2020', '2021', '2022', '2023']
tabla['Años']=anios
tabla['Precisión']=Lprecision
tabla['Exactitud']=Lexactitud
tabla['Sensibilidad']=Lsensibilidad
tabla = tabla.set_index('Años')
tabla

,Precisión,Exactitud,Sensibilidad
Años,,,
2020,0.598039,0.571286,0.135556
2021,0.603922,0.542261,0.327660
2022,0.583333,0.532300,0.130319
2023,0.666667,0.566667,0.175439
